In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/UDEA-Esp-Analitica-y-Ciencia-de-Datos/EACD-03-BIGDATA/master/init.py
import init; init.init(force_download=False); 
from IPython.display import Image

replicating local resources


In [ ]:
Image("local/imgs/udea-datascience.png")

# **TALLER 3: DATAFRAME**
## Desarrollado por: Mg. Efraín Alberto Oviedo
## Universidad de Antioquia

Por favor, ingrese la siguiente información:

**Nombre Completo:July Andrea Muñoz Lopera**

**Cédula: 1020393965**

**Correo Institucional: july.munoz@udea.edu.co**

**Nombre Completo:Oscar Felipe Robayo Lozano**

**Cédula: 1234191132**

**Correo Institucional:oscar.robayo1303@gmail.com**

**Nombre Completo: Daniel Alberto Lopez Sanchez**

**Cédula: 1036668134**

**Correo Institucional: daniel.lopez6@udea.edu.co**


#**Pacientes sometidos a cirugías por tumores malignos**

Se dispone de un conjunto de datos (pacientes.json) que contiene datos socio demográficos de pacientes que fueron sometidos a procedimientos quirúrgicos durante los años 2007 a 2018 y cuyos Códigos Internacionales de Enfermedades (CIE-10) correspondía a tumores malignos. Tomado de: https://www.datos.gov.co/Salud-y-Protecci-n-Social/Perfil-socio-demogr-fico-pacientes-sometidos-a-cir/i45c-f4fs


Por cada paciente, se dispone de los siguientes atributos:
 
* DiagnosticoAgrupado: Nombre del Grupo de Diagnóstico Agrupado
* Cirugia: Descripción del Procedimiento Quirúrgico
* TipoAtencion: Tipo de atención que generó el procedimiento (Urgencia-Programada)
* Edad: Edad en años del paciente
* Subregion: Subregión del departamento de la cual proviene el paciente
* Regimen: Régimen de salud al cual pertenecía el paciente al momento del procedimiento
* Genero: Genero del paciente
* Estado_Civil: Estado civil del paciente al momento del procedimiento quirúrgico
* Etnia: Etnia a la cual pertenecía el paciente al momento del procedimiento quirúrgico
* Estrato: Nivel socio económico del paciente al momento del procedimiento quirúrgico
* Año en el cual fue realizado el procedimiento quirúrgico

Se requiere de su apoyo para comprender el comportamiento de esta enfermedad, respondiendo las siguientes inquietudes:



In [9]:
#Instalación
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz 
!pip install -q findspark

#Variables de Entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

#SparkContext
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

from pyspark.sql.types import *

In [4]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

In [14]:
pacientes = sqlCtx.read.option("header",True) \
     .json("local/data/pacientes.json")
pacientes.show(20)

+----+--------------------+--------------------+----+------------+---------+--------+------+------------+--------------+------------+
| Año|             Cirugia| DiagnosticoAgrupado|Edad|Estado_Civil|  Estrato|   Etnia|Genero|     Regimen|     Subregion|TipoAtencion|
+----+--------------------+--------------------+----+------------+---------+--------+------+------------+--------------+------------+
|2007|   TRAQUEOSTOMIA SOD|CARCINOMA IN SITU...|  77|      Casado|Estrato 4|Indigena|     M|  PARTICULAR|    Centro Sur|    Urgencia|
|2007|   TRAQUEOSTOMIA SOD|CARCINOMA IN SITU...|  77|      Casado|Estrato 4|Indigena|     M|  PARTICULAR|    Centro Sur|    Urgencia|
|2007|MICROENDOSCOPIA L...|CARCINOMA IN SITU...|  77|      Casado|Estrato 4|Indigena|     M|  PARTICULAR|    Centro Sur|    Urgencia|
|2007|MICROENDOSCOPIA L...|CARCINOMA IN SITU...|  77|      Casado|Estrato 4| Ninguna|     M|  PARTICULAR|    Centro Sur|    Urgencia|
|2007|HEMICOLECTOMIA DE...|TUMOR MALIGNO DEL...|  66|      Cas

##**A) Calidad de datos**

Revise cada una de las variables disponibles y solucione los posibles problemas presentes respecto a registros nulos y atípicos.

Analice el tipo de dato utilizado para cada variable y realice los ajustes que considere necesarios para optimizar el almacenamiento

Justificando las decisiones tomadas en cada caso 

Presente aquí el código desarrollado y el resultado obtenido

In [15]:
pacientes.printSchema()

root
 |-- Año: long (nullable = true)
 |-- Cirugia: string (nullable = true)
 |-- DiagnosticoAgrupado: string (nullable = true)
 |-- Edad: long (nullable = true)
 |-- Estado_Civil: string (nullable = true)
 |-- Estrato: string (nullable = true)
 |-- Etnia: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Regimen: string (nullable = true)
 |-- Subregion: string (nullable = true)
 |-- TipoAtencion: string (nullable = true)



Podemos visualizar los tipos de datos asumidos para cada variable de la base. Identificamos que los tipos string están correctamente tipificados. Pero por ejemplo Año y Edad, que son los datos númericos, podemos optimizarlos cambiando el tipo de dato.

In [11]:
schema = StructType() \
      .add("Año",IntegerType(),True) \
      .add("Cirugia",StringType(),True) \
      .add("DiagnosticoAgrupado",StringType(),True) \
      .add("Edad",ByteType(),True) \
      .add("Estado_Civil",StringType(),True) \
      .add("Estrato",StringType(),True) \
      .add("Etnia",StringType(),True) \
      .add("Genero",StringType(),True) \
      .add("Regimen",StringType(),True) \
      .add("Subregion",StringType(),True) \
      .add("TipoAtencion",StringType(),True)

In [12]:
pacientes_df = sqlCtx.read.format("json") \
      .option("header", True) \
      .option("delimiter", ",") \
      .schema(schema) \
      .load("local/data/pacientes.json")

In [13]:
pacientes_df.printSchema()
pacientes_df.show()

root
 |-- Año: integer (nullable = true)
 |-- Cirugia: string (nullable = true)
 |-- DiagnosticoAgrupado: string (nullable = true)
 |-- Edad: byte (nullable = true)
 |-- Estado_Civil: string (nullable = true)
 |-- Estrato: string (nullable = true)
 |-- Etnia: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Regimen: string (nullable = true)
 |-- Subregion: string (nullable = true)
 |-- TipoAtencion: string (nullable = true)

+----+--------------------+--------------------+----+------------+---------+--------+------+------------+--------------+------------+
| Año|             Cirugia| DiagnosticoAgrupado|Edad|Estado_Civil|  Estrato|   Etnia|Genero|     Regimen|     Subregion|TipoAtencion|
+----+--------------------+--------------------+----+------------+---------+--------+------+------------+--------------+------------+
|2007|   TRAQUEOSTOMIA SOD|CARCINOMA IN SITU...|  77|      Casado|Estrato 4|Indigena|     M|  PARTICULAR|    Centro Sur|    Urgencia|
|2007|   TRAQUEO

##**B) Análisis por rango de edad**

Identifique las 5 cirugías mas frecuentes para los mayores de 80 años y para los menores de 20 años, indicando la influencia del género para cada grupo de edad 

Presente aquí el código desarrollado y el resultado obtenido

##**C) Análisis por diagnóstico**

Identifique los cinco tipos de diagnóstico más comunes y para ellos presente el promedio de edad por género y Subregión

Presente aquí el código desarrollado y el resultado obtenido

##**D) Análisis por estrato y estado civil**

Identifique para el total de pacientes, cuáles son los 5 tipos de cirugías más frecuentes. Para cada tipo de cirugía calcule el porcentaje de ocurrencia para las variables estrato y estado civil

Presente aquí el código desarrollado y el resultado obtenido

##**E) Seguimiento de las cirugías en el tiempo**

Para los cinco tipos de cirugías más frecuentes, verifique si los casos han aumentado o disminuido en los últimos cinco años.
Para cada cirugía presente el promedio de edad de los pacientes por género en cada año

Presente aquí el código desarrollado y el resultado obtenido

#**Notas:**

* Analice los resultados de cada punto y presente sus conclusiones
* Utilice al menos una vez cada uno de los siguientes métodos

  *   agg
  *   withColumn
  *   groupBy
  *   sql
  *   registerFunction

